# Movie Review Prediction

In [1]:
from nltk.corpus import movie_reviews
import random
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.corpus import stopwords
import string
import nltk
from nltk import NaiveBayesClassifier

### Loading Data.....

In [2]:
movie_reviews.categories()

['neg', 'pos']

In [3]:
print(len(movie_reviews.fileids()))
movie_reviews.fileids()

2000


['neg/cv000_29416.txt',
 'neg/cv001_19502.txt',
 'neg/cv002_17424.txt',
 'neg/cv003_12683.txt',
 'neg/cv004_12641.txt',
 'neg/cv005_29357.txt',
 'neg/cv006_17022.txt',
 'neg/cv007_4992.txt',
 'neg/cv008_29326.txt',
 'neg/cv009_29417.txt',
 'neg/cv010_29063.txt',
 'neg/cv011_13044.txt',
 'neg/cv012_29411.txt',
 'neg/cv013_10494.txt',
 'neg/cv014_15600.txt',
 'neg/cv015_29356.txt',
 'neg/cv016_4348.txt',
 'neg/cv017_23487.txt',
 'neg/cv018_21672.txt',
 'neg/cv019_16117.txt',
 'neg/cv020_9234.txt',
 'neg/cv021_17313.txt',
 'neg/cv022_14227.txt',
 'neg/cv023_13847.txt',
 'neg/cv024_7033.txt',
 'neg/cv025_29825.txt',
 'neg/cv026_29229.txt',
 'neg/cv027_26270.txt',
 'neg/cv028_26964.txt',
 'neg/cv029_19943.txt',
 'neg/cv030_22893.txt',
 'neg/cv031_19540.txt',
 'neg/cv032_23718.txt',
 'neg/cv033_25680.txt',
 'neg/cv034_29446.txt',
 'neg/cv035_3343.txt',
 'neg/cv036_18385.txt',
 'neg/cv037_19798.txt',
 'neg/cv038_9781.txt',
 'neg/cv039_5963.txt',
 'neg/cv040_8829.txt',
 'neg/cv041_22364.txt',


In [4]:
movie_reviews.fileids("pos")

['pos/cv000_29590.txt',
 'pos/cv001_18431.txt',
 'pos/cv002_15918.txt',
 'pos/cv003_11664.txt',
 'pos/cv004_11636.txt',
 'pos/cv005_29443.txt',
 'pos/cv006_15448.txt',
 'pos/cv007_4968.txt',
 'pos/cv008_29435.txt',
 'pos/cv009_29592.txt',
 'pos/cv010_29198.txt',
 'pos/cv011_12166.txt',
 'pos/cv012_29576.txt',
 'pos/cv013_10159.txt',
 'pos/cv014_13924.txt',
 'pos/cv015_29439.txt',
 'pos/cv016_4659.txt',
 'pos/cv017_22464.txt',
 'pos/cv018_20137.txt',
 'pos/cv019_14482.txt',
 'pos/cv020_8825.txt',
 'pos/cv021_15838.txt',
 'pos/cv022_12864.txt',
 'pos/cv023_12672.txt',
 'pos/cv024_6778.txt',
 'pos/cv025_3108.txt',
 'pos/cv026_29325.txt',
 'pos/cv027_25219.txt',
 'pos/cv028_26746.txt',
 'pos/cv029_18643.txt',
 'pos/cv030_21593.txt',
 'pos/cv031_18452.txt',
 'pos/cv032_22550.txt',
 'pos/cv033_24444.txt',
 'pos/cv034_29647.txt',
 'pos/cv035_3954.txt',
 'pos/cv036_16831.txt',
 'pos/cv037_18510.txt',
 'pos/cv038_9749.txt',
 'pos/cv039_6170.txt',
 'pos/cv040_8276.txt',
 'pos/cv041_21113.txt',
 

In [5]:
print(movie_reviews.fileids()[5])
movie_reviews.words(movie_reviews.fileids()[5])

neg/cv005_29357.txt


['capsule', ':', 'in', '2176', 'on', 'the', 'planet', ...]

### Cleaning

In [6]:
documents = []
for category in movie_reviews.categories():
    for fileid in movie_reviews.fileids(category):
        documents.append((movie_reviews.words(fileid), category))


print(type(documents[0]))
print("withoutshuffle\n\n" , documents[0 : 5] , "\n")
random.shuffle(documents)
print("after_shuffle\n\n" , documents[0 : 5])


<class 'tuple'>
withoutshuffle

 [(['plot', ':', 'two', 'teen', 'couples', 'go', 'to', ...], 'neg'), (['the', 'happy', 'bastard', "'", 's', 'quick', 'movie', ...], 'neg'), (['it', 'is', 'movies', 'like', 'these', 'that', 'make', ...], 'neg'), (['"', 'quest', 'for', 'camelot', '"', 'is', 'warner', ...], 'neg'), (['synopsis', ':', 'a', 'mentally', 'unstable', 'man', ...], 'neg')] 

after_shuffle

 [(['it', "'", 's', 'not', 'a', 'bad', 'thing', 'to', ...], 'neg'), (['i', 'came', 'to', 'an', 'epiphany', 'while', ...], 'neg'), (['what', 'makes', 'reindeer', 'games', 'even', 'more', ...], 'neg'), (['spoiled', 'rich', 'kid', 'kelley', 'morse', '(', ...], 'neg'), (['since', '1990', ',', 'the', 'dramatic', 'picture', ...], 'pos')]


In [7]:
lemmatizer = WordNetLemmatizer()

def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.ADV
    else :
        return wordnet.NOUN
## set is to just sort the list and to take a word as feature only a single time
stops = set(stopwords.words('english'))
print(string.punctuation)
punctuations = list(string.punctuation)
stops.update(punctuations)
stops 

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


{'!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'need

In [8]:
def clean_review(words):
    output_words = []
    for w in words:
        if w.lower() not in stops:
            pos = pos_tag([w])
            clean_word = lemmatizer.lemmatize(w, pos = get_simple_pos(pos[0][1]))
            output_words.append(clean_word.lower())
    return output_words

In [9]:
documents = [(clean_review(document), category) for document, category in documents]

In [10]:
documents[0]

(['bad',
  'thing',
  'update',
  'old',
  'stories',
  'do',
  'time',
  'oral',
  'tradition',
  'difficulty',
  'film',
  'tale',
  'change',
  'dry',
  'new',
  'cinderella',
  'danielle',
  'drew',
  'barrymore',
  'sit',
  'around',
  'wait',
  'prince',
  'come',
  'woman',
  'late',
  'twentieth',
  'century',
  'albeit',
  'situate',
  'sixteenth',
  'century',
  'france',
  'father',
  'dies',
  'stays',
  'house',
  'even',
  'though',
  'mistreat',
  'wicked',
  'stepmother',
  'rodmilla',
  'anjelica',
  'huston',
  'works',
  'day',
  'reads',
  'sir',
  'thomas',
  'firelight',
  'night',
  'although',
  'story',
  'update',
  'know',
  'plot',
  'danielle',
  'meets',
  'prince',
  'henry',
  'dougray',
  'scott',
  'go',
  'ball',
  'disguise',
  'later',
  'found',
  'live',
  'happily',
  'ever',
  'movie',
  'often',
  'like',
  'watch',
  'film',
  'play',
  'theater',
  'real',
  'life',
  'engross',
  'theater',
  'screen',
  'usually',
  'lifeless',
  'seem',
  

### Split the data

In [11]:
training_documents = documents[0 : 1500]
testing_documents = documents[1500 : ]

### Building Features Set

In [12]:
all_words = []
for doc in training_documents:
    all_words += doc[0]
freq = nltk.FreqDist(all_words)
print(type(freq)) ## Frequency distribution objects
common = freq.most_common(3000)
features = [i[0] for i in common]
features

<class 'nltk.probability.FreqDist'>


['film',
 'movie',
 'one',
 'make',
 'like',
 'get',
 'go',
 'see',
 'even',
 'time',
 'good',
 'take',
 'story',
 'would',
 'much',
 'character',
 'also',
 'come',
 'bad',
 'give',
 'two',
 'well',
 'characters',
 'first',
 'seem',
 '--',
 'way',
 'life',
 'know',
 'films',
 'plot',
 'end',
 'say',
 'really',
 'little',
 'people',
 'look',
 'could',
 'man',
 'scene',
 'great',
 'never',
 'work',
 'best',
 'big',
 'new',
 'scenes',
 'love',
 'many',
 'want',
 'director',
 'movies',
 'u',
 'action',
 'watch',
 'another',
 'back',
 'show',
 'still',
 'something',
 'act',
 'star',
 'world',
 'however',
 'try',
 'think',
 'use',
 'every',
 'old',
 'though',
 'audience',
 'real',
 'around',
 'enough',
 'year',
 'last',
 'cast',
 'find',
 'role',
 'better',
 'performance',
 'may',
 'young',
 'long',
 'comedy',
 'years',
 'write',
 'john',
 'interest',
 'play',
 'script',
 'thing',
 'funny',
 'things',
 'fact',
 'although',
 'actually',
 'nothing',
 'almost',
 'played',
 'since',
 'ever',
 'r

In [13]:
def get_feature_dict(words):
    current_features = {}
    words_set = set(words)
    for w in features:
        current_features[w] = w in words_set
    return current_features

In [14]:
output = get_feature_dict(training_documents[0][0])
output

{'film': True,
 'movie': True,
 'one': True,
 'make': False,
 'like': True,
 'get': True,
 'go': True,
 'see': False,
 'even': True,
 'time': True,
 'good': True,
 'take': False,
 'story': True,
 'would': False,
 'much': True,
 'character': False,
 'also': False,
 'come': True,
 'bad': True,
 'give': False,
 'two': False,
 'well': True,
 'characters': True,
 'first': False,
 'seem': True,
 '--': False,
 'way': False,
 'life': True,
 'know': True,
 'films': True,
 'plot': True,
 'end': False,
 'say': False,
 'really': False,
 'little': False,
 'people': False,
 'look': False,
 'could': False,
 'man': True,
 'scene': False,
 'great': False,
 'never': False,
 'work': False,
 'best': False,
 'big': False,
 'new': True,
 'scenes': False,
 'love': True,
 'many': False,
 'want': False,
 'director': False,
 'movies': False,
 'u': False,
 'action': False,
 'watch': True,
 'another': False,
 'back': False,
 'show': False,
 'still': False,
 'something': True,
 'act': True,
 'star': True,
 'world'

In [15]:
training_data = [(get_feature_dict(doc), category) for doc, category in training_documents]
testing_data = [(get_feature_dict(doc), category) for doc, category in training_documents]

In [16]:
training_data[0]

({'film': True,
  'movie': True,
  'one': True,
  'make': False,
  'like': True,
  'get': True,
  'go': True,
  'see': False,
  'even': True,
  'time': True,
  'good': True,
  'take': False,
  'story': True,
  'would': False,
  'much': True,
  'character': False,
  'also': False,
  'come': True,
  'bad': True,
  'give': False,
  'two': False,
  'well': True,
  'characters': True,
  'first': False,
  'seem': True,
  '--': False,
  'way': False,
  'life': True,
  'know': True,
  'films': True,
  'plot': True,
  'end': False,
  'say': False,
  'really': False,
  'little': False,
  'people': False,
  'look': False,
  'could': False,
  'man': True,
  'scene': False,
  'great': False,
  'never': False,
  'work': False,
  'best': False,
  'big': False,
  'new': True,
  'scenes': False,
  'love': True,
  'many': False,
  'want': False,
  'director': False,
  'movies': False,
  'u': False,
  'action': False,
  'watch': True,
  'another': False,
  'back': False,
  'show': False,
  'still': False

### Classification using NLTK Naive Bayes

In [17]:
clf = NaiveBayesClassifier.train(training_data)

In [18]:
nltk.classify.accuracy(clf, testing_data)

0.89

In [19]:
clf.show_most_informative_features(15)

Most Informative Features
                  seagal = True              neg : pos    =     11.3 : 1.0
             outstanding = True              pos : neg    =      8.9 : 1.0
            breathtaking = True              pos : neg    =      7.6 : 1.0
                   anger = True              pos : neg    =      6.2 : 1.0
                   mulan = True              pos : neg    =      6.2 : 1.0
             wonderfully = True              pos : neg    =      6.1 : 1.0
              schumacher = True              neg : pos    =      6.0 : 1.0
                ordinary = True              pos : neg    =      5.9 : 1.0
                     8mm = True              neg : pos    =      5.8 : 1.0
                flawless = True              pos : neg    =      5.7 : 1.0
              ridiculous = True              neg : pos    =      5.6 : 1.0
                  spacey = True              pos : neg    =      5.6 : 1.0
               innocence = True              pos : neg    =      5.4 : 1.0

### Using Sklearn Classifier within NLTK

In [20]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from nltk.classify.scikitlearn import SklearnClassifier

In [21]:
## SVC
svc = SVC()
classifier_sklearn = SklearnClassifier(svc)
classifier_sklearn.train(training_data)
nltk.classify.accuracy(classifier_sklearn , testing_data)

0.9953333333333333

In [22]:
## Random Forest
rfc = RandomForestClassifier()
classifier_sklearn1 = SklearnClassifier(rfc)
classifier_sklearn1.train(training_data)
nltk.classify.accuracy(classifier_sklearn1 , testing_data)

1.0

### Count Vectorizer ( convert the data into the format that sklearn required)

#### a) On Raw data

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
## There is an option in count vectorizer(stop_words) 
## which takes list of stop words and can do the work for us.

In [24]:
train_set = {"the sky sky is blue" , "the sun is bright"}
count_vec = CountVectorizer(max_features = 3)
a = count_vec.fit_transform(train_set)
print(a.todense())
a

[[1 0 1]
 [1 2 1]]


<2x3 sparse matrix of type '<class 'numpy.int64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [25]:
count_vec.get_feature_names()

['is', 'sky', 'the']

In [26]:
a = ["pagal" , "and", "darpokk"]
" ".join(a)

'pagal and darpokk'

#### b) on movie datasets

In [27]:
documents[0]

(['bad',
  'thing',
  'update',
  'old',
  'stories',
  'do',
  'time',
  'oral',
  'tradition',
  'difficulty',
  'film',
  'tale',
  'change',
  'dry',
  'new',
  'cinderella',
  'danielle',
  'drew',
  'barrymore',
  'sit',
  'around',
  'wait',
  'prince',
  'come',
  'woman',
  'late',
  'twentieth',
  'century',
  'albeit',
  'situate',
  'sixteenth',
  'century',
  'france',
  'father',
  'dies',
  'stays',
  'house',
  'even',
  'though',
  'mistreat',
  'wicked',
  'stepmother',
  'rodmilla',
  'anjelica',
  'huston',
  'works',
  'day',
  'reads',
  'sir',
  'thomas',
  'firelight',
  'night',
  'although',
  'story',
  'update',
  'know',
  'plot',
  'danielle',
  'meets',
  'prince',
  'henry',
  'dougray',
  'scott',
  'go',
  'ball',
  'disguise',
  'later',
  'found',
  'live',
  'happily',
  'ever',
  'movie',
  'often',
  'like',
  'watch',
  'film',
  'play',
  'theater',
  'real',
  'life',
  'engross',
  'theater',
  'screen',
  'usually',
  'lifeless',
  'seem',
  

In [28]:
categories = [category for document , category in documents]
categories

['neg',
 'neg',
 'neg',
 'neg',
 'pos',
 'neg',
 'neg',
 'neg',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'neg',
 'neg',
 'neg',
 'pos',
 'neg',
 'pos',
 'neg',
 'pos',
 'pos',
 'pos',
 'neg',
 'neg',
 'neg',
 'pos',
 'pos',
 'pos',
 'pos',
 'neg',
 'neg',
 'pos',
 'pos',
 'pos',
 'neg',
 'neg',
 'neg',
 'pos',
 'neg',
 'neg',
 'neg',
 'pos',
 'pos',
 'neg',
 'neg',
 'pos',
 'neg',
 'neg',
 'pos',
 'neg',
 'neg',
 'neg',
 'pos',
 'pos',
 'pos',
 'neg',
 'neg',
 'neg',
 'neg',
 'neg',
 'pos',
 'pos',
 'pos',
 'neg',
 'neg',
 'neg',
 'neg',
 'pos',
 'neg',
 'neg',
 'neg',
 'pos',
 'neg',
 'pos',
 'neg',
 'neg',
 'pos',
 'neg',
 'pos',
 'neg',
 'neg',
 'neg',
 'neg',
 'pos',
 'neg',
 'pos',
 'neg',
 'pos',
 'neg',
 'pos',
 'neg',
 'pos',
 'pos',
 'neg',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'pos',
 'neg',
 'pos',
 'neg',
 'neg',
 'pos',
 'neg',
 'pos',
 'neg',
 'neg',
 'pos',
 'pos',
 'neg',
 'neg',
 'neg',
 'pos',
 'neg',
 'neg',
 'pos',
 'pos',
 'neg',
 'neg',
 'neg',
 'pos',


In [29]:
text_documents = [" ".join(document) for document , category in documents]
text_documents

['bad thing update old stories do time oral tradition difficulty film tale change dry new cinderella danielle drew barrymore sit around wait prince come woman late twentieth century albeit situate sixteenth century france father dies stays house even though mistreat wicked stepmother rodmilla anjelica huston works day reads sir thomas firelight night although story update know plot danielle meets prince henry dougray scott go ball disguise later found live happily ever movie often like watch film play theater real life engross theater screen usually lifeless seem much go camera star films co conspirator danielle convictions compromise socialist feminist pretends royalty get man kind woman last act movie one revenge enjoyable characters cartoonish fairy godmother stand leonardo da vinci patrick godfrey fun watch eccentric old man advises danielle huston enjoyable top wickedness fairness companion love film maybe one chick flicks men like certainly date film nearly entire audience couple

In [30]:
from sklearn.model_selection import train_test_split

In [31]:
x_train, x_test, y_train, y_test = train_test_split(text_documents, categories)

In [32]:
count_vec = CountVectorizer(max_features = 2000 , ngram_range = (1 , 2))
x_train_feature = count_vec.fit_transform(x_train)
x_train_feature.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 1, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 2, 0]], dtype=int64)

In [33]:
count_vec.get_feature_names()

['000',
 '10',
 '100',
 '13',
 '17',
 '1995',
 '1996',
 '1997',
 '1998',
 '1999',
 '20',
 '30',
 '50',
 '80',
 '90',
 'abandon',
 'ability',
 'able',
 'absolutely',
 'academy',
 'accent',
 'accept',
 'accident',
 'accompany',
 'accomplish',
 'achieve',
 'across',
 'act',
 'action',
 'action film',
 'action scenes',
 'action sequences',
 'actions',
 'actor',
 'actors',
 'actress',
 'acts',
 'actual',
 'actually',
 'adam',
 'adams',
 'adapt',
 'adaptation',
 'add',
 'addition',
 'admit',
 'adult',
 'adults',
 'adventure',
 'affair',
 'affleck',
 'age',
 'agent',
 'ago',
 'agree',
 'ahead',
 'air',
 'al',
 'alan',
 'alex',
 'alice',
 'alien',
 'aliens',
 'alive',
 'allen',
 'allow',
 'allows',
 'almost',
 'alone',
 'along',
 'along way',
 'already',
 'also',
 'although',
 'always',
 'amaze',
 'america',
 'american',
 'among',
 'amount',
 'amuse',
 'amy',
 'anderson',
 'angel',
 'angels',
 'angry',
 'animal',
 'animals',
 'animate',
 'animation',
 'anne',
 'annie',
 'annoy',
 'another',
 '

In [34]:
x_test_feature = count_vec.transform(x_test)
x_test_feature

<500x2000 sparse matrix of type '<class 'numpy.int64'>'
	with 84434 stored elements in Compressed Sparse Row format>

### Sklearn Classifier

In [35]:
from sklearn.svm import SVC
svc = SVC()
svc.fit(x_train_feature , y_train)
svc.score(x_test_feature , y_test)

0.81